<a href="https://colab.research.google.com/github/jhonaraujos/TCC_UNDB/blob/main/Modelo_Generalizado_Para_as_Rodas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime

# Carregar os dados do CSV
df_original = pd.read_csv("/content/EVO - Degradaçao de Largura de Friso.csv", delimiter=';')

# Selecionar as colunas desejadas
colunas_desejadas = ["Train Time", "Vehicle Tag", "Vehicle Axle Number", "Side of Vehicle", "Site",  "Flange Thickness (mm)", "Flange Thickness Confidence",
"Flange Height (mm)", "Flange Height Confidence", "Rim Thickness (mm)", "Rim Thickness Confidence","Tread Hollow (mm)",
                     "Tread Hollow Confidence","Flange Slope (mm)", "BTB (mm)", "Wheel Diameter (mm)", "Rail Dist (mm)"]
df_original= df_original[colunas_desejadas]
# Remover duplicatas considerando todas as colunas
df_original = df_original.drop_duplicates()
# Filtrar o DataFrame para manter apenas as linhas onde "Flange Thickness (mm)" não é vazio e com medida com confiança e somente da Linha 1
df = df_original.loc[df_original["Site"] != "16K888 Lina 3"].copy()
# Criar a coluna "Date REF" com AM ou PM e remover AM/PM da coluna "Train Time"
df["Date REF"] = df["Train Time"].str.extract(r'(\bAM\b|\bPM\b)', expand=False)
df["Train Time"] = df["Train Time"].str.replace(r'\s*(AM|PM)\b', '', regex=True)
# Função para converter o formato de data/hora
def converter_train_time(data_hora, date_ref):
    try:
        if date_ref in ['AM', 'PM']:
            # Tentar converter datas no formato com AM/PM
            data_hora = datetime.strptime(data_hora, '%m/%d/%Y %I:%M:%S')
            if date_ref == 'PM' and data_hora.hour != 12:
                data_hora = data_hora.replace(hour=data_hora.hour + 12)
            elif date_ref == 'AM' and data_hora.hour == 12:
                data_hora = data_hora.replace(hour=0)
        else:
            # Tentar converter datas no formato padrão sem AM/PM
            try:
                data_hora = datetime.strptime(data_hora, '%m/%d/%Y %H:%M')
            except ValueError:
                data_hora = datetime.strptime(data_hora, '%m/%d/%Y %H:%M:%S')
        return data_hora  # Retorna como objeto datetime
    except ValueError:
        return None
# Garantir que 'Train Time' é datetime para ordenação correta
df["Train Time"] = df.apply(lambda row: converter_train_time(row["Train Time"], row["Date REF"]), axis=1)
df['Train Time'] = pd.to_datetime(df['Train Time'], errors='coerce')
# Criar a coluna "id" com a junção das colunas Numero do veiculo + Numero do Rodeiro + Lado do Rodeiro (L = LEFT , R = RIGHT)
df["id"] = df["Vehicle Tag"] + "_" + df["Vehicle Axle Number"].astype(str) + "_" + df["Side of Vehicle"]
# Ordenar o DataFrame por "Train Time" em ordem crescente e depois por "Vehicle Axle Number"
df = df.sort_values(by=["Train Time", "Vehicle Axle Number"], ascending=[True, True])
# Criar a coluna "passagem" que conta sequencialmente cada valor único na coluna "id"
df["passagem"] = df.groupby('id').cumcount() + 1
# Ordenar novamente para exibir na ordem correta das passagens de forma crescente
df = df.sort_values(by=["id", "passagem"])
df

,Train Time,Vehicle Tag,Vehicle Axle Number,Side of Vehicle,Site,Flange Thickness (mm),Flange Thickness Confidence,Flange Height (mm),Flange Height Confidence,Rim Thickness (mm),Rim Thickness Confidence,Tread Hollow (mm),Tread Hollow Confidence,Flange Slope (mm),BTB (mm),Wheel Diameter (mm),Rail Dist (mm),Date REF,id,passagem
19452,2023-07-17 23:02:08,EVO 2001,1,L,16K888 Lina 1,25.4,Yes,30.1,Yes,29.4,Yes,0.0,Yes,1.5,1515.7,1002.0,44.3,PM,EVO 2001_1_L,1
16716,2023-07-23 07:52:43,EVO 2001,1,L,16K888 Lina 1,25.4,Yes,30.3,Yes,29.2,Yes,0.0,Yes,2.6,1516.1,1001.0,44.7,AM,EVO 2001_1_L,2
16260,2023-07-24 07:22:45,EVO 2001,1,L,16K888 Lina 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,EVO 2001_1_L,3
15456,2023-07-27 12:50:55,EVO 2001,1,L,16K888 Lina 1,25.4,Yes,30.3,Yes,29.7,Yes,0.0,Yes,1.5,1515.3,1001.0,45.2,PM,EVO 2001_1_L,4
13920,2023-07-31 01:47:02,EVO 2001,1,L,16K888 Lina 1,25.3,Yes,30.4,Yes,29.4,Yes,0.0,Yes,1.4,1516.8,1000.0,42.2,AM,EVO 2001_1_L,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25911,2024-04-20 06:12:06,EVO 298,6,R,16K888 Lina 1,30.9,Yes,29.0,Yes,64.0,Yes,0.9,Yes,1.9,1516.3,1073.0,44.7,AM,EVO 298_6_R,79
78243,2024-05-01 12:39:00,EVO 298,6,R,16K888 Lina 1,31.4,Yes,29.4,Yes,63.6,Yes,0.5,Yes,2.0,1517.9,1070.0,42.9,NaN,EVO 298_6_R,80
22491,2024-05-15 06:42:37,EVO 298,6,R,16K888 Lina 1,27.1,Yes,29.1,Yes,63.4,Yes,0.4,No,5.4,1518.2,1071.0,43.3,AM,EVO 298_6_R,81
21639,2024-05-19 06:22:07,EVO 298,6,R,16K888 Lina 1,31.2,Yes,29.7,Yes,63.3,Yes,0.3,Yes,1.7,1517.9,1066.0,43.0,AM,EVO 298_6_R,82


In [2]:
import pandas as pd

# Reordenar o dataframe por 'id' e 'passagem' em ordem crescente para facilitar o preenchimento
df_sorted = df.sort_values(by=['id', 'passagem'], ascending=[True, True])

# Lista de colunas para aplicar a lógica de preenchimento
columns_to_fill = ["Flange Thickness (mm)", "Flange Height (mm)", "Rim Thickness (mm)",
                   "Tread Hollow (mm)", "Flange Slope (mm)", "BTB (mm)", "Wheel Diameter (mm)", "Rail Dist (mm)"]

# Função para preencher os valores nulos com a média dos últimos 3 valores anteriores, incluindo os valores substituídos
def fill_na_with_mean(group):
    for column in columns_to_fill:
        filled_values = group[column].copy()
        for i in range(len(filled_values)):
            if pd.isna(filled_values.iloc[i]):
                # Pegar os últimos 3 valores anteriores
                previous_values = filled_values.iloc[max(0, i-3):i]
                if not previous_values.empty:
                    mean_value = previous_values.mean()
                else:
                    # Se não houver valores anteriores, usar o próximo valor não nulo
                    mean_value = filled_values.iloc[i+1:].dropna().iloc[0]
                filled_values.iloc[i] = mean_value
        group[column] = filled_values
    return group

# Aplicar a função para cada grupo de 'id'
df_filled = df_sorted.groupby('id', group_keys=False).apply(fill_na_with_mean)

# Função para arredondar para baixo para uma casa decimal
def round_down_one_decimal(value):
    return (value * 10 // 1) / 10

# Aplicar a função de arredondamento para as colunas selecionadas
for column in columns_to_fill:
    df_filled[column] = df_filled[column].apply(round_down_one_decimal)

# Verificação para um ID específico
specific_id = 'EVO 2001_1_L'
original_na_specific = df[(df['id'] == specific_id) & (df[columns_to_fill].isna().any(axis=1))]
filled_na_specific_corrected = df_filled[(df_filled['id'] == specific_id) & (df_filled.index.isin(original_na_specific.index))]

original_na_specific_output = original_na_specific[['id', 'passagem'] + columns_to_fill]
filled_na_specific_corrected_output = filled_na_specific_corrected[['id', 'passagem'] + columns_to_fill]

display(original_na_specific_output, filled_na_specific_corrected_output)


,id,passagem,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),BTB (mm),Wheel Diameter (mm),Rail Dist (mm)
16260,EVO 2001_1_L,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13680,EVO 2001_1_L,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5088,EVO 2001_1_L,22,23.8,31.1,27.9,0.3,0.8,1517.0,NaN,41.7
57228,EVO 2001_1_L,26,32.3,28.4,NaN,0.0,1.5,1519.1,NaN,42.3
90062,EVO 2001_1_L,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38868,EVO 2001_1_L,55,29.3,27.5,35.8,0.0,2.0,1517.4,NaN,49.0
37704,EVO 2001_1_L,57,29.3,27.3,35.3,0.0,1.2,1518.2,NaN,49.9
76756,EVO 2001_1_L,59,29.4,27.3,35.3,0.0,0.5,1517.0,NaN,50.5
71536,EVO 2001_1_L,60,28.9,27.4,35.0,0.0,1.1,1517.7,NaN,50.7
66132,EVO 2001_1_L,61,28.6,27.3,NaN,0.0,2.0,1516.4,NaN,51.3


,id,passagem,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),BTB (mm),Wheel Diameter (mm),Rail Dist (mm)
16260,EVO 2001_1_L,3,25.4,30.2,29.3,0.0,2.0,1515.9,1001.5,44.5
13680,EVO 2001_1_L,6,25.3,30.3,29.4,0.0,1.6,1516.0,1000.8,43.9
5088,EVO 2001_1_L,22,23.8,31.1,27.9,0.3,0.8,1517.0,995.3,41.7
57228,EVO 2001_1_L,26,32.3,28.4,27.6,0.0,1.5,1519.1,995.0,42.3
90062,EVO 2001_1_L,36,31.1,28.6,35.7,0.0,2.2,1516.2,1010.6,45.8
38868,EVO 2001_1_L,55,29.3,27.5,35.8,0.0,2.0,1517.4,1010.3,49.0
37704,EVO 2001_1_L,57,29.3,27.3,35.3,0.0,1.2,1518.2,1010.4,49.9
76756,EVO 2001_1_L,59,29.4,27.3,35.3,0.0,0.5,1517.0,1009.1,50.5
71536,EVO 2001_1_L,60,28.9,27.4,35.0,0.0,1.1,1517.7,1008.8,50.7
66132,EVO 2001_1_L,61,28.6,27.3,35.1,0.0,2.0,1516.4,1008.3,51.3


In [3]:


# Lista de colunas para aplicar a lógica de preenchimento
columns_to_fill = ["Flange Thickness (mm)", "Flange Height (mm)", "Rim Thickness (mm)",
                   "Tread Hollow (mm)", "Flange Slope (mm)", "BTB (mm)", "Wheel Diameter (mm)", "Rail Dist (mm)"]

# Função para preencher os valores com a média dos últimos 3 valores anteriores, incluindo os valores substituídos, quando o 'Site' for '16K888 Lina 2'
def fill_with_mean_for_specific_site(group):
    for column in columns_to_fill:
        filled_values = group[column].copy()
        for i in range(len(filled_values)):
            if group.iloc[i]['Site'] == '16K888 Lina 2':
                # Pegar os últimos 3 valores anteriores
                previous_values = filled_values.iloc[max(0, i-3):i]
                if not previous_values.empty:
                    mean_value = previous_values.mean()
                else:
                    # Se não houver valores anteriores, usar o próximo valor não nulo
                    mean_value = filled_values.iloc[i+1:].dropna().iloc[0]
                filled_values.iloc[i] = mean_value
        group[column] = filled_values
    return group

# Aplicar a função para cada grupo de 'id' na base já tratada pelo primeiro código
df_filled_site_specific = df_filled.groupby('id', group_keys=False).apply(fill_with_mean_for_specific_site)

# Função para arredondar para baixo para uma casa decimal
def round_down_one_decimal(value):
    return (value * 10 // 1) / 10

# Aplicar a função de arredondamento para as colunas selecionadas
for column in columns_to_fill:
    df_filled_site_specific[column] = df_filled_site_specific[column].apply(round_down_one_decimal)

# Verificação para um ID específico
specific_id = 'EVO 2001_1_L'
original_na_specific_site = df_filled[(df_filled['id'] == specific_id) & (df_filled['Site'] == '16K888 Lina 2')]
filled_na_specific_corrected_site = df_filled_site_specific[(df_filled_site_specific['id'] == specific_id) & (df_filled_site_specific['Site'] == '16K888 Lina 2') & (df_filled_site_specific.index.isin(original_na_specific_site.index))]

original_na_specific_site_output = original_na_specific_site[['id', 'passagem', 'Site'] + columns_to_fill]
filled_na_specific_corrected_site_output = filled_na_specific_corrected_site[['id', 'passagem', 'Site'] + columns_to_fill]

display(original_na_specific_site_output, filled_na_specific_corrected_site_output)



,id,passagem,Site,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),BTB (mm),Wheel Diameter (mm),Rail Dist (mm)
16260,EVO 2001_1_L,3,16K888 Lina 2,25.4,30.2,29.3,0.0,2.0,1515.9,1001.5,44.5
13680,EVO 2001_1_L,6,16K888 Lina 2,25.3,30.3,29.4,0.0,1.6,1516.0,1000.8,43.9
90062,EVO 2001_1_L,36,16K888 Lina 2,31.1,28.6,35.7,0.0,2.2,1516.2,1010.6,45.8


,id,passagem,Site,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),BTB (mm),Wheel Diameter (mm),Rail Dist (mm)
16260,EVO 2001_1_L,3,16K888 Lina 2,25.4,30.2,29.3,0.0,2.0,1515.9,1001.5,44.5
13680,EVO 2001_1_L,6,16K888 Lina 2,25.3,30.3,29.4,0.0,1.6,1516.0,1000.8,43.9
90062,EVO 2001_1_L,36,16K888 Lina 2,31.1,28.6,35.7,0.0,2.2,1516.2,1010.6,45.8


In [4]:
import pandas as pd

# Lista de colunas para aplicar a lógica de preenchimento
columns_to_fill = ["Flange Thickness (mm)", "Flange Height (mm)", "Rim Thickness (mm)",
                   "Tread Hollow (mm)", "Flange Slope (mm)", "BTB (mm)", "Wheel Diameter (mm)", "Rail Dist (mm)"]

# Função para preencher os valores com a média dos últimos 3 valores anteriores, incluindo os valores substituídos, quando o 'Flange Thickness Confidence' for 'No'
def fill_with_mean_for_confidence_no(group):
    for column in columns_to_fill:
        filled_values = group[column].copy()
        for i in range(len(filled_values)):
            if group.iloc[i]['Flange Thickness Confidence'] == 'No':
                # Pegar os últimos 3 valores anteriores
                previous_values = filled_values.iloc[max(0, i-3):i]
                if not previous_values.empty:
                    mean_value = previous_values.mean()
                else:
                    # Se não houver valores anteriores, usar o próximo valor não nulo
                    mean_value = filled_values.iloc[i+1:].dropna().iloc[0]
                filled_values.iloc[i] = mean_value
        group[column] = filled_values
    return group

# Aplicar a função para cada grupo de 'id' na base já tratada pelos dois códigos anteriores
df_filled_confidence_no = df_filled_site_specific.groupby('id', group_keys=False).apply(fill_with_mean_for_confidence_no)

# Função para arredondar para baixo para uma casa decimal
def round_down_one_decimal(value):
    return (value * 10 // 1) / 10

# Aplicar a função de arredondamento para as colunas selecionadas
for column in columns_to_fill:
    df_filled_confidence_no[column] = df_filled_confidence_no[column].apply(round_down_one_decimal)

# Verificação para um ID específico e 'Flange Thickness Confidence' igual a 'No'
specific_id = 'EVO 2001_3_L'
original_na_specific_confidence = df_filled_site_specific[(df_filled_site_specific['id'] == specific_id) & (df_filled_site_specific['Flange Thickness Confidence'] == 'No')]
filled_na_specific_corrected_confidence = df_filled_confidence_no[(df_filled_confidence_no['id'] == specific_id) & (df_filled_confidence_no['Flange Thickness Confidence'] == 'No') & (df_filled_confidence_no.index.isin(original_na_specific_confidence.index))]

original_na_specific_confidence_output = original_na_specific_confidence[['id', 'passagem', 'Flange Thickness Confidence'] + columns_to_fill]
filled_na_specific_corrected_confidence_output = filled_na_specific_corrected_confidence[['id', 'passagem', 'Flange Thickness Confidence'] + columns_to_fill]

display(original_na_specific_confidence_output, filled_na_specific_corrected_confidence_output)


,id,passagem,Flange Thickness Confidence,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),BTB (mm),Wheel Diameter (mm),Rail Dist (mm)
27544,EVO 2001_3_L,85,No,29.5,30.1,32.3,2.2,1.4,1518.9,1006.0,37.2


,id,passagem,Flange Thickness Confidence,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),BTB (mm),Wheel Diameter (mm),Rail Dist (mm)
27544,EVO 2001_3_L,85,No,31.0,28.9,31.5,2.3,2.0,1515.4,1004.3,37.6


In [5]:
# Salvar o dataframe final para continuar os tratamentos
df_final = df_filled_confidence_no.copy()

# Mostrar as primeiras linhas do dataframe final para verificação
df_final

,Train Time,Vehicle Tag,Vehicle Axle Number,Side of Vehicle,Site,Flange Thickness (mm),Flange Thickness Confidence,Flange Height (mm),Flange Height Confidence,Rim Thickness (mm),Rim Thickness Confidence,Tread Hollow (mm),Tread Hollow Confidence,Flange Slope (mm),BTB (mm),Wheel Diameter (mm),Rail Dist (mm),Date REF,id,passagem
19452,2023-07-17 23:02:08,EVO 2001,1,L,16K888 Lina 1,25.4,Yes,30.1,Yes,29.4,Yes,0.0,Yes,1.5,1515.7,1002.0,44.3,PM,EVO 2001_1_L,1
16716,2023-07-23 07:52:43,EVO 2001,1,L,16K888 Lina 1,25.4,Yes,30.3,Yes,29.2,Yes,0.0,Yes,2.6,1516.1,1001.0,44.7,AM,EVO 2001_1_L,2
16260,2023-07-24 07:22:45,EVO 2001,1,L,16K888 Lina 2,25.4,NaN,30.2,NaN,29.3,NaN,0.0,NaN,2.0,1515.9,1001.5,44.5,AM,EVO 2001_1_L,3
15456,2023-07-27 12:50:55,EVO 2001,1,L,16K888 Lina 1,25.4,Yes,30.3,Yes,29.7,Yes,0.0,Yes,1.5,1515.3,1001.0,45.2,PM,EVO 2001_1_L,4
13920,2023-07-31 01:47:02,EVO 2001,1,L,16K888 Lina 1,25.3,Yes,30.4,Yes,29.4,Yes,0.0,Yes,1.4,1516.8,1000.0,42.2,AM,EVO 2001_1_L,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25911,2024-04-20 06:12:06,EVO 298,6,R,16K888 Lina 1,30.9,Yes,29.0,Yes,64.0,Yes,0.9,Yes,1.9,1516.3,1073.0,44.7,AM,EVO 298_6_R,79
78243,2024-05-01 12:39:00,EVO 298,6,R,16K888 Lina 1,31.4,Yes,29.4,Yes,63.6,Yes,0.5,Yes,2.0,1517.9,1070.0,42.9,NaN,EVO 298_6_R,80
22491,2024-05-15 06:42:37,EVO 298,6,R,16K888 Lina 1,27.1,Yes,29.1,Yes,63.4,Yes,0.4,No,5.4,1518.2,1071.0,43.3,AM,EVO 298_6_R,81
21639,2024-05-19 06:22:07,EVO 298,6,R,16K888 Lina 1,31.2,Yes,29.7,Yes,63.3,Yes,0.3,Yes,1.7,1517.9,1066.0,43.0,AM,EVO 298_6_R,82


In [6]:
df_final = df_final.drop(['Wheel Diameter (mm)','Rail Dist (mm)','BTB (mm)','Train Time','Vehicle Tag','Vehicle Axle Number','Side of Vehicle','Site','Flange Height Confidence','Flange Thickness Confidence','Rim Thickness Confidence','Tread Hollow Confidence','Date REF'], axis=1)
df_final

,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),id,passagem
19452,25.4,30.1,29.4,0.0,1.5,EVO 2001_1_L,1
16716,25.4,30.3,29.2,0.0,2.6,EVO 2001_1_L,2
16260,25.4,30.2,29.3,0.0,2.0,EVO 2001_1_L,3
15456,25.4,30.3,29.7,0.0,1.5,EVO 2001_1_L,4
13920,25.3,30.4,29.4,0.0,1.4,EVO 2001_1_L,5
...,...,...,...,...,...,...,...
25911,30.9,29.0,64.0,0.9,1.9,EVO 298_6_R,79
78243,31.4,29.4,63.6,0.5,2.0,EVO 298_6_R,80
22491,27.1,29.1,63.4,0.4,5.4,EVO 298_6_R,81
21639,31.2,29.7,63.3,0.3,1.7,EVO 298_6_R,82


In [7]:
import numpy as np

# Função para aplicar as operações para cada grupo de ID
def apply_operations(group):
    group['MM5d'] = group['Flange Thickness (mm)'].rolling(window=5).mean().round(2)
    group['MM10d'] = group['Flange Thickness (mm)'].rolling(window=10).mean().round(2)
    group['MM30d'] = group['Flange Thickness (mm)'].rolling(window=30).mean().round(2)

    # Mudança percentual na espessura da flange em relação ao valor anterior
    group['Flange_Thickness_Change'] = group['Flange Thickness (mm)'].pct_change()

    # Substitui os valores infinitos (causados pela divisão por zero) por NaN
    group['Flange_Thickness_Change'].replace([np.inf, -np.inf], np.nan, inplace=True)

    # Preencher NaN com 0 na mudança percentual
    group['Flange_Thickness_Change'].fillna(0, inplace=True)

    # Função para preencher valores em branco nas médias móveis com a média dos valores anteriores
    def fill_na_with_previous_mean(series):
        for i in range(len(series)):
            if pd.isna(series.iloc[i]):
                if i == 0:
                    series.iloc[i] = group['Flange Thickness (mm)'].iloc[i]
                else:
                    series.iloc[i] = group['Flange Thickness (mm)'].iloc[:i].mean()
        return series

    group['MM5d'] = fill_na_with_previous_mean(group['MM5d'])
    group['MM10d'] = fill_na_with_previous_mean(group['MM10d'])
    group['MM30d'] = fill_na_with_previous_mean(group['MM30d'])

    return group

# Aplicar a função para cada grupo de 'id'
df_final = df_final.groupby('id', group_keys=False).apply(apply_operations)

# Primeiro, agrupe os dados por 'id'
groups = df_final.groupby('id')

# Defina uma função para calcular a degradação por passagem
def calculate_degradation(row, first_value):
    return (row['Flange Thickness (mm)'] - first_value) / row['passagem']

# Aplique essa função a cada grupo
for name, group in groups:
    first_value = group.iloc[0]['Flange Thickness (mm)']
    df_final.loc[group.index, 'Degradation per Passage'] = group.apply(calculate_degradation, axis=1, first_value=first_value)



df_final.head()

,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),id,passagem,MM5d,MM10d,MM30d,Flange_Thickness_Change,Degradation per Passage
19452,25.4,30.1,29.4,0.0,1.5,EVO 2001_1_L,1,25.40,25.4,25.4,0.000000,0.00
16716,25.4,30.3,29.2,0.0,2.6,EVO 2001_1_L,2,25.40,25.4,25.4,0.000000,0.00
16260,25.4,30.2,29.3,0.0,2.0,EVO 2001_1_L,3,25.40,25.4,25.4,0.000000,0.00
15456,25.4,30.3,29.7,0.0,1.5,EVO 2001_1_L,4,25.40,25.4,25.4,0.000000,0.00
13920,25.3,30.4,29.4,0.0,1.4,EVO 2001_1_L,5,25.38,25.4,25.4,-0.003937,-0.02


In [8]:
# Fazer uma cópia do DataFrame final
df_shifted = df_final.copy()

# Aplicar o shift de -10 na coluna 'Flange Thickness (mm)' para cada grupo de 'id'
df_shifted['Flange Thickness (mm)'] = df_shifted.groupby('id')['Flange Thickness (mm)'].shift(-20)

# Excluir os valores em branco resultantes do shift
df_shifted = df_shifted.dropna(subset=['Flange Thickness (mm)'])


df_shifted.head()


,Flange Thickness (mm),Flange Height (mm),Rim Thickness (mm),Tread Hollow (mm),Flange Slope (mm),id,passagem,MM5d,MM10d,MM30d,Flange_Thickness_Change,Degradation per Passage
19452,24.5,30.1,29.4,0.0,1.5,EVO 2001_1_L,1,25.40,25.4,25.4,0.000000,0.00
16716,23.8,30.3,29.2,0.0,2.6,EVO 2001_1_L,2,25.40,25.4,25.4,0.000000,0.00
16260,23.8,30.2,29.3,0.0,2.0,EVO 2001_1_L,3,25.40,25.4,25.4,0.000000,0.00
15456,23.8,30.3,29.7,0.0,1.5,EVO 2001_1_L,4,25.40,25.4,25.4,0.000000,0.00
13920,23.8,30.4,29.4,0.0,1.4,EVO 2001_1_L,5,25.38,25.4,25.4,-0.003937,-0.02


In [38]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression

# Remover a coluna 'id' do DataFrame final
X = df_final.drop(columns=['id', 'Flange Thickness (mm)'])
y = df_final['Flange Thickness (mm)']

# Aplicar SelectKBest para selecionar as melhores features
best_features = SelectKBest(score_func=f_regression, k='all')
fit = best_features.fit(X, y)

# Obter os scores das features
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

# Concatenar os scores e os nomes das colunas
feature_scores = pd.concat([dfcolumns, dfscores], axis=1)
feature_scores.columns = ['Feature', 'Score']

# Ordenar as features pelo score
best_features_sorted = feature_scores.sort_values(by='Score', ascending=False)

# Exibir as melhores features
best_features_sorted.head(10)  # Mostra as 10 melhores features


,Feature,Score
5,MM5d,1.104347e+06
6,MM10d,5.391151e+05
7,MM30d,2.410186e+05
0,Flange Height (mm),1.176161e+04
9,Degradation per Passage,1.093751e+04
2,Tread Hollow (mm),6.356178e+03
3,Flange Slope (mm),4.002042e+03
1,Rim Thickness (mm),3.317216e+03
8,Flange_Thickness_Change,2.630899e+03
4,passagem,2.848493e+02


In [9]:
!pip install pycaret

In [10]:
import pandas as pd
from pycaret.regression import setup, compare_models, pull, predict_model

# Selecionar 5 ids únicos aleatórios
unique_ids = df_shifted['id'].drop_duplicates().sample(500).values

# Combinar os dados dos 5 IDs aleatórios em um único DataFrame
df_combined = df_shifted[df_shifted['id'].isin(unique_ids)]

# Configurar PyCaret para regressão com dados combinados
reg = setup(data=df_combined, target='Flange Thickness (mm)', verbose=False)

# Comparar os modelos e selecionar o melhor
best_model = compare_models()

# Puxar o dataframe com os resultados de comparação
results = pull()
print("Best model overall:")
print(results.head())

# Inicializar um dicionário para armazenar as métricas de desempenho para cada id
performance_metrics = {}

# Avaliar o melhor modelo separadamente para cada ID
for id_value in unique_ids:
    df_id = df_shifted[df_shifted['id'] == id_value]
    predictions = predict_model(best_model, data=df_id)

    # Extrair métricas de desempenho
    metrics = predictions[['id', 'Flange Thickness (mm)', 'prediction_label']]
    performance_metrics[id_value] = metrics

    print(f"\nPerformance metrics for ID {id_value}:")
    print(metrics.head())

# Mostrar as métricas de desempenho para cada id
for id_value, metrics in performance_metrics.items():
    print(f"\nPerformance metrics for ID {id_value}:")
    print(metrics.head())



,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.4407,0.5519,0.7422,0.8900,0.0244,0.0148,7.4190
rf,Random Forest Regressor,0.4965,0.6337,0.7953,0.8736,0.0262,0.0167,20.2790
xgboost,Extreme Gradient Boosting,0.6436,0.8737,0.9344,0.8258,0.0308,0.0217,0.4520
lightgbm,Light Gradient Boosting Machine,0.7143,1.0364,1.0177,0.7933,0.0336,0.0241,1.1020
dt,Decision Tree Regressor,0.6260,1.2284,1.1073,0.7547,0.0365,0.0210,0.4460
knn,K Neighbors Regressor,0.7764,1.2406,1.1135,0.7525,0.0368,0.0262,0.1550
gbr,Gradient Boosting Regressor,0.8314,1.3931,1.1800,0.7222,0.0393,0.0282,4.5730
ridge,Ridge Regression,0.9092,1.6711,1.2925,0.6668,0.0431,0.0309,0.1660
br,Bayesian Ridge,0.9092,1.6714,1.2926,0.6668,0.0431,0.0309,0.0990
lr,Linear Regression,0.9092,1.6711,1.2925,0.6668,0.0431,0.0309,1.5270


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

Best model overall:
                                    Model     MAE     MSE    RMSE      R2  \
et                  Extra Trees Regressor  0.4407  0.5519  0.7422  0.8900   
rf                Random Forest Regressor  0.4965  0.6337  0.7953  0.8736   
xgboost         Extreme Gradient Boosting  0.6436  0.8737  0.9344  0.8258   
lightgbm  Light Gradient Boosting Machine  0.7143  1.0364  1.0177  0.7933   
dt                Decision Tree Regressor  0.6260  1.2284  1.1073  0.7547   

           RMSLE    MAPE  TT (Sec)  
et        0.0244  0.0148     7.419  
rf        0.0262  0.0167    20.279  
xgboost   0.0308  0.0217     0.452  
lightgbm  0.0336  0.0241     1.102  
dt        0.0365  0.0210     0.446  


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.1623,0.1926,0.4389,0.7656,0.0132,0.0051



Performance metrics for ID EVO 258_6_R:
                id  Flange Thickness (mm)  prediction_label
19691  EVO 258_6_R              31.299999         31.299999
19379  EVO 258_6_R              31.299999         31.299999
18095  EVO 258_6_R              31.200001         31.200001
17099  EVO 258_6_R              31.200001         31.218000
81243  EVO 258_6_R              31.000000         31.000000


...


Performance metrics for ID EVO 264_2_R:
                id  Flange Thickness (mm)  prediction_label
19203  EVO 264_2_R              28.799999         28.799999
18723  EVO 264_2_R              29.200001         29.200001
17271  EVO 264_2_R              29.299999         29.299999
16083  EVO 264_2_R              29.400000         29.301000
14619  EVO 264_2_R              29.400000         29.301000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.1171,0.0691,0.2629,0.8879,0.0079,0.0036



Performance metrics for ID EVO 285_3_R:
                id  Flange Thickness (mm)  prediction_label
19097  EVO 285_3_R              33.400002         33.500000
18617  EVO 285_3_R              33.500000         33.500000
17321  EVO 285_3_R              33.500000         33.500000
16697  EVO 285_3_R              33.599998         33.526000
16133  EVO 285_3_R              33.599998         33.531999


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.2663,0.5803,0.7618,0.5562,0.0237,0.0084



Performance metrics for ID EVO 293_5_L:
                id  Flange Thickness (mm)  prediction_label
19820  EVO 293_5_L              31.400000         31.400000
15548  EVO 293_5_L              31.299999         31.299999
15224  EVO 293_5_L              31.600000         31.600000
14144  EVO 293_5_L              31.600000         31.600000
96624  EVO 293_5_L              31.799999         31.799999

Performance metrics for ID EVO 258_6_R:
                id  Flange Thickness (mm)  prediction_label
19691  EVO 258_6_R              31.299999         31.299999
19379  EVO 258_6_R              31.299999         31.299999
18095  EVO 258_6_R              31.200001         31.200001
17099  EVO 258_6_R              31.200001         31.218000
81243  EVO 258_6_R              31.000000         31.000000

Performance metrics for ID EVO 295_6_L:
                id  Flange Thickness (mm)  prediction_label
18586  EVO 295_6_L              32.599998         32.599998
18178  EVO 295_6_L              32.59